In [ ]:
# --- Imports ---

import os
import sys
import sqlite3
import pandas as pd

sys.path.append('..')
from database_manager import DatabaseManager

In [ ]:
# Load databases

db_manager = DatabaseManager()

# For Voyage 3.5 model
df_queries = db_manager.load_embeddings_from_sql(db_path="../../databases/voyage_3.5_nice_guidelines_2048.db")

# For Voyage 3 Large model
# df_queries = db_manager.load_embeddings_from_sql(db_path="../../databases/voyage_3_large_nice_guidelines_2048.db")

# For OpenAI text-embedding-3-large
# df_queries = db_manager.load_embeddings_from_sql(db_path="../../databases/text_embedding_3_large_nice_guidelines.db")

# For Qwen3
# df_queries = db_manager.load_embeddings_from_sql(db_path="../../databases/Qwen3-Embedding-0.6B_nice_guidelines.db")

print(f"Loaded {len(df_queries)} queries")
df_queries.head()

In [ ]:
# Add guideline urls to database

def add_url_column_to_db(db_path):
    """Add URL column to the chunks table in the database"""
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        cursor.execute("PRAGMA table_info(chunks)")
        columns = [column[1] for column in cursor.fetchall()]
        
        if 'url' not in columns:
            cursor.execute("ALTER TABLE chunks ADD COLUMN url TEXT")
            
            cursor.execute("""
                UPDATE chunks 
                SET url = 'https://www.nice.org.uk/guidance/' || LOWER(source)
            """)
            
            conn.commit()
            print(f"Added URL column to {db_path}")
        else:
            print(f"URL column already exists in {db_path}")
            
    except Exception as e:
        print(f"Error modifying {db_path}: {e}")
    finally:
        conn.close()

databases = [
    "../../databases/voyage_3.5_nice_guidelines_2048.db",
    "../../databases/voyage_3_large_nice_guidelines_2048.db", 
    "../../databases/text_embedding_3_large_nice_guidelines.db",
    "../../databases/Qwen3-Embedding-0.6B_nice_guidelines.db"
]

for db_path in databases:
    if os.path.exists(db_path):
        add_url_column_to_db(db_path)
    else:
        print(f"Database not found: {db_path}")